# Libraries

In [36]:
import json, requests, os, time, glob
import pandas as pd

from io import StringIO
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

#import undetected_chromedriver as uc
#from datetime import datetime, timedelta
from tqdm.notebook import tqdm


# Open the browser

In [2]:
#driver = uc.Chrome()
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [3]:
initial_url = 'https://www.sec.gov/search-filings/cik-lookup'

KEYWORD = 'APPLE'

# W/O Wait

In [22]:
driver.get(initial_url)

driver.find_element(By.ID, 'company').send_keys(KEYWORD)
driver.find_element(By.XPATH, '//input[@id="company"]/following-sibling::button').click()

# Wait

In [ ]:
driver.get(initial_url)

In [ ]:
element = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'company'))
)

# Fill the search box

In [23]:
driver.find_element(By.ID, 'company').send_keys(KEYWORD)
driver.find_element(By.XPATH, '//input[@id="company"]/following-sibling::button').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="company"]"}
  (Session info: chrome=129.0.6668.90); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00937143+25587]
	(No symbol) [0x008CA2E4]
	(No symbol) [0x007C2113]
	(No symbol) [0x00806F62]
	(No symbol) [0x008071AB]
	(No symbol) [0x00847852]
	(No symbol) [0x0082ABE4]
	(No symbol) [0x00845370]
	(No symbol) [0x0082A936]
	(No symbol) [0x007FBA73]
	(No symbol) [0x007FC4CD]
	GetHandleVerifier [0x00C14C63+3030803]
	GetHandleVerifier [0x00C66B99+3366473]
	GetHandleVerifier [0x009C95F2+624802]
	GetHandleVerifier [0x009D0E6C+655644]
	(No symbol) [0x008D2C9D]
	(No symbol) [0x008CFD68]
	(No symbol) [0x008CFF05]
	(No symbol) [0x008C2336]
	BaseThreadInitThunk [0x764C7BA9+25]
	RtlInitializeExceptionChain [0x7796C11B+107]
	RtlClearBits [0x7796C09F+191]

# Scrape EDGAR CIK

In [24]:
ciks = driver.find_elements(By.XPATH, '//a[contains(@href, "browse-edgar")]')
#ciks_name = driver.find_elements(By.XPATH, '//pre[2]/text()[1]')
#ciks_name = driver.find_elements(By.XPATH, '//a[contains(@href, "browse-edgar")]/following-sibling::text[1]')

#//a[contains(@href, "browse-edgar")]/following-sibling::text()[1]

In [34]:
len(ciks)

100

In [27]:
urls = []
cik_list = []
for cik in ciks:
    new_url = 'https://www.sec.gov/edgar/browse/?CIK=' + cik.text
    urls.append(new_url)

    if cik.text:
        cik_list.append(cik.text)
    

In [34]:
#urls

In [14]:
S = requests.session()
for cookie in driver.get_cookies(): S.cookies.set(cookie['name'], cookie['value'])

In [37]:
new_dir = 'sec_cik_data'

if not os.path.exists(new_dir): os.makedirs(new_dir)

files = glob.glob(os.path.join(new_dir, '*'))
for file in files:
    try: os.remove(file)
    except: pass

In [49]:
new_dir_filing = 'sec_filing_data'

if not os.path.exists(new_dir_filing): os.makedirs(new_dir_filing)

files = glob.glob(os.path.join(new_dir_filing, '*'))
for file in files:
    try: os.remove(file)
    except: pass

In [50]:
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Accept-Language': 'en-US,en;q=0.5',
    'Origin': 'https://www.sec.gov',
    'Referer': 'https://www.sec.gov/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:131.0) Gecko/20100101 Firefox/131.0'
}

for cik in cik_list[:1]:
    api_url = 'https://data.sec.gov/submissions/CIK' + cik + '.json'
    print(api_url)
    r = S.get(api_url, headers=headers)
    #print(r.json())

    if r.json():
        filename = os.path.join(new_dir, cik + '.json')
        with open(filename, 'w') as f:
            json.dump(r.json(), f, indent=4)

        data = r.json()
        document_urls = []
        if 'filings' in data and 'recent' in data['filings']:
            filings = data['filings']['recent']
            accessions = filings['accessionNumber']
            documents = filings['primaryDocument']
            for _ in range(len(accessions)):
                doc_url = 'https://www.sec.gov/Archives/edgar/data/' + cik + '/' + accessions[_].replace('-', '') + '/' + documents[_]
                document_urls.append(doc_url)

                rr = S.get(doc_url, headers=headers)
                doc_filename = os.path.join(new_dir_filing, accessions[_].replace('-', '') + '_' + documents[_].replace('/', '_'))
                with open(doc_filename, 'wb') as df:
                    df.write(rr.content)


https://data.sec.gov/submissions/CIK0001493086.json


In [12]:
for url in urls[:1]:
    print(url)
    driver.get(url)

    try:
        btnFillings = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//button[@id="btnViewAllFilings"]'))
        )
        driver.find_element(By.XPATH, '//button[@id="btnViewAllFilings"]').click()
    except:
        pass
    time.sleep(5)

https://www.sec.gov/edgar/browse/?CIK=0001493086
https://www.sec.gov/edgar/browse/?CIK=0001653690
https://www.sec.gov/edgar/browse/?CIK=0001567043
https://www.sec.gov/edgar/browse/?CIK=0001412692
https://www.sec.gov/edgar/browse/?CIK=0001839531
